# Load packages and functions

In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
import tensorflow_addons as tfa
import tensorflow as tf

import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory|

import warnings
warnings.filterwarnings('ignore')

base_dir = "G:/Mon Drive/" 


In [10]:
import tensorflow.keras.backend as K
from tensorflow.keras.losses import CategoricalCrossentropy


class WeightedCategoricalCrossentropy(CategoricalCrossentropy):

    def __init__(self, cost_mat=np.ones((5, 5)), name='weighted_categorical_crossentropy', **kwargs):
        assert(cost_mat.ndim == 2)
        assert(cost_mat.shape[0] == cost_mat.shape[1])

        super().__init__(name=name, **kwargs)
        self.cost_mat = K.cast_to_floatx(cost_mat)

    def __call__(self, y_true, y_pred, sample_weight=None):

        return super().__call__(
            y_true=y_true,
            y_pred=y_pred,
            sample_weight=get_sample_weights(y_true, y_pred, self.cost_mat),
        )
        
    def get_config(self):
        return super().get_config().copy().update(
            {'cost_mat': self.cost_mat}
        )
    #def get_config(self):
    #    return {'cost_mat': self.cost_mat}
        
    @classmethod
    def from_config(cls, config):
       return cls(**config)


def get_sample_weights(y_true, y_pred, cost_m):
    num_classes = len(cost_m)

    y_pred.shape.assert_has_rank(2)
    assert(y_pred.shape[1] == num_classes)
    y_pred.shape.assert_is_compatible_with(y_true.shape)

    y_pred = K.one_hot(K.argmax(y_pred), num_classes)

    y_true_nk1 = K.expand_dims(y_true, 2)
    y_pred_n1k = K.expand_dims(y_pred, 1)
    cost_m_1kk = K.expand_dims(cost_m, 0)

    sample_weights_nkk = cost_m_1kk * y_true_nk1 * y_pred_n1k
    sample_weights_n = K.sum(sample_weights_nkk, axis=[1, 2])

    return sample_weights_n


# Categorical accuracy:
from tensorflow.keras.metrics import CategoricalAccuracy


class WeightedCategoricalAccuracy(CategoricalAccuracy):

    def __init__(self, cost_mat=np.ones((5, 5)), name='weighted_categorical_accuracy', **kwargs):
        assert(cost_mat.ndim == 2)
        assert(cost_mat.shape[0] == cost_mat.shape[1])

        super().__init__(name=name, **kwargs)
        self.cost_mat = K.cast_to_floatx(cost_mat)

    def update_state(self, y_true, y_pred, sample_weight=None):

        return super().update_state(
            y_true=y_true,
            y_pred=y_pred,
            sample_weight=get_sample_weights(y_true, y_pred, self.cost_mat),
        )
# model.compile(metrics=[WeightedCategoricalAccuracy(cost_matrix), ...], ...)

# Matriz de costos: 
import numpy as np
w_array = np.ones((5, 5))
# If >1 - it penalize Class i classified as j
i = 0
j = 1 
w_array[i, j] = 1
# Overall enphasis in class i:
w_array[1, :] = 5 #10
w_array[2, :] = 5 #5
w_array[3, :] = 10 #29
w_array[4, :] = 10 #37
cost_mat = w_array
cost_mat


array([[ 1.,  1.,  1.,  1.,  1.],
       [ 5.,  5.,  5.,  5.,  5.],
       [ 5.,  5.,  5.,  5.,  5.],
       [10., 10., 10., 10., 10.],
       [10., 10., 10., 10., 10.]])

#### Calculate Sensibility and Precision

In [11]:
def eval_class_result(new_model, thisclass=1):
    selection_v1_1 = Y_valid[:,thisclass]==1
    x = X_valid[selection_v1_1]
    y_oh =  Y_valid[selection_v1_1]
    y = np.argmax(y_oh, axis = 1)
    p_pre = new_model.predict(x, batch_size = 10)
    y_pre = np.argmax(p_pre, axis = 1)
    if np.sum(y_pre==thisclass)==0:
        tpr=0
    else:
        tpr = np.sum(y_pre==y)/np.sum(y_pre==thisclass)
    sens = np.sum(y_pre==y)/np.sum(y==thisclass)

    return(np.round(sens*100,1),np.round(tpr*100,1))
 

def eval_class_result_all2(new_model):
    
    print("class | sensibility | precision")
    
    x = X_valid
    y_oh =  Y_valid
    y = np.argmax(y_oh, axis = 1)
    p_pre = new_model.predict(x, batch_size = 10)
    y_pre = np.argmax(p_pre, axis = 1)
    
    for thisclass in range(5):
        if np.sum(y_pre==thisclass)==0:
            tpr=0
        else:
            tpr = np.sum(np.logical_and(y_pre==y, y_pre==thisclass))/np.sum(y_pre==thisclass)
        sens = np.sum(np.logical_and(y_pre==y, y==thisclass))/np.sum(y==thisclass)
        print((thisclass, np.round(sens*100,1),np.round(tpr*100,1)))
    

    return True


def eval_class_result_all(new_model):
    print("sens | true-pos-rate")
    print(eval_class_result(new_model, thisclass=0))
    print(eval_class_result(new_model, thisclass=1))
    print(eval_class_result(new_model, thisclass=2))
    print(eval_class_result(new_model, thisclass=3))
    print(eval_class_result(new_model, thisclass=4))

    return True

    #:::::::::::::::::::::::::::::::::::

def eval_class_result3(Y_valid, Y_pred_valid, thisclass):
    
    print("class | sens | true-pos-rate")
    y_oh =  Y_valid
    y = np.argmax(y_oh, axis = 1)
    #p_pre = new_model.predict(x, batch_size = 10)
    y_pre = Y_pred_valid #np.argmax(p_pre, axis = 1)
    
    for thisclass in range(5):
        if np.sum(y_pre==thisclass)==0:
            tpr=0
        else:
            tpr = np.sum(np.logical_and(y_pre==y, y_pre==thisclass))/np.sum(y_pre==thisclass)
        sens = np.sum(np.logical_and(y_pre==y, y==thisclass))/np.sum(y==thisclass)
        print((thisclass, np.round(sens*100,1),np.round(tpr*100,1)))
    

    return True

def eval_class_result_all3(Y_valid, Y_pred_valid):
    
    print("class | sens | true-pos-rate")
    
    x = X_valid
    y_oh =  Y_valid
    y = np.argmax(y_oh, axis = 1)
    y_pre = Y_pred_valid
    
    for thisclass in range(5):
        if np.sum(y_pre==thisclass)==0:
            tpr=0
        else:
            tpr = np.sum(np.logical_and(y_pre==y, y_pre==thisclass))/np.sum(y_pre==thisclass)
        sens = np.sum(np.logical_and(y_pre==y, y==thisclass))/np.sum(y==thisclass)
        print((thisclass, np.round(sens*100,1),np.round(tpr*100,1)))
    

    return True


### Calculate credible intervals to asses stability

In [12]:
def eval_model_distrib(new_model, Niter = 100, sample_size = 200):
    #Niter = 100
    losses = []
    for i in range(Niter):
        # random sample:
        #sample_size = 200
        selection_v1_1 = np.random.randint(0,Y_valid.shape[0], size=sample_size)
        x = X_valid[selection_v1_1]
        y_oh =  Y_valid[selection_v1_1]
        y = np.argmax(y_oh, axis = 1)
        p_pre = new_model.predict(x, batch_size = 10)
        y_pre = np.argmax(p_pre, axis = 1)
        y_pre_oh = np.zeros((y_pre.size, 5))
        y_pre_oh[np.arange(y_pre.size),y_pre] = 1
        # Calculate loss:
        kappa_loss = tfa.losses.WeightedKappaLoss(num_classes=5, weightage="quadratic")
        aux = kappa_loss(tf.constant(y_oh), tf.constant(y_pre_oh)).numpy()
        losses.append(-aux)
        
        IC95 = np.quantile(losses, [0.025, 0.975])
        median = np.quantile(losses, [0.5])
        mean = np.mean(losses)

    print("WeightedKappaLoss: Median + IC95")
    print(median)
    print(IC95)

    return True

def eval_model_distrib3(Y_valid, Y_pred_valid, Niter = 100, sample_size = 200):
    #Niter = 100
    losses = []
    for i in range(Niter):
        # random sample:
        #sample_size = 200
        selection_v1_1 = np.random.randint(0,Y_valid.shape[0], size=sample_size)
        y_oh =  Y_valid[selection_v1_1]
        y = np.argmax(y_oh, axis = 1)
        y_pre = Y_pred_valid[selection_v1_1]
        y_pre_oh = np.zeros((y_pre.size, 5))
        y_pre_oh[np.arange(y_pre.size), y_pre] = 1
        # Calculate loss:
        kappa_loss = tfa.losses.WeightedKappaLoss(num_classes=5, weightage="quadratic")
        aux = kappa_loss(tf.constant(y_oh), tf.constant(y_pre_oh)).numpy()
        losses.append(-aux)

        IC95 = np.quantile(losses, [0.025, 0.975])
        median = np.quantile(losses, [0.5])
        mean = np.mean(losses)

    print("WeightedKappaLoss: Median + IC95")
    print(median)
    print(IC95)

    return True


In [13]:
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('INFO')
tf.autograph.set_verbosity(0)
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

# Evaluation

## Load data 320x320

In [14]:
im_size1=320
im_size2=320
X_valid = np.load('x_modeltesting_'+str(im_size1)+'.npy', mmap_mode='r')
Y_valid = np.load('y_modeltesting_'+str(im_size1)+'.npy', mmap_mode='r') 
Y_valid = np.array(Y_valid, np.float32)

## Evaluate models

### Modelos EfficientNetB3

#### IMSize=100

In [33]:
modelname = "EfficientNetB3_v3softmax.12--11.72"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Convertir modelo para que acepte input de 320
initmodelsize = 100
inputs = tf.keras.Input(shape=(320, 320, 3))
x = tf.keras.layers.Resizing(initmodelsize,initmodelsize)(inputs)
outputs = new_model(x, training=False)
finalmodel = tf.keras.Model(inputs, outputs)
#finalmodel.save(base_dir + 'models/PretrainedNetworks/'+ modelname + '_im320.h5') # if saved

# Evaluation:
pred_valid = finalmodel.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)

138/138 [==============================] - 11s 61ms/step
class | sens | true-pos-rate
(0, 100.0, 73.1)
(1, 0.0, 0)
(2, 0.0, 0)
(3, 0.0, 0)
(4, 0.0, 0)
WeightedKappaLoss: Median + IC95
[-9.53673862e-07]
[-9.53673862e-07 -9.53673862e-07]


True

In [15]:
modelname = "EfficientNetB0_Augm_im150.18--0.59"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Convertir modelo para que acepte input de 320
initmodelsize = 150
inputs = tf.keras.Input(shape=(320, 320, 3))
x = tf.keras.layers.Resizing(initmodelsize,initmodelsize)(inputs)
outputs = new_model(x, training=False)
finalmodel = tf.keras.Model(inputs, outputs)
#finalmodel.save(base_dir + 'models/PretrainedNetworks/'+ modelname + '_im320.h5') # if saved

# Evaluation:
pred_valid = finalmodel.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)

138/138 [==============================] - 61s 430ms/step
class | sens | true-pos-rate
(0, 96.0, 74.9)
(1, 2.6, 4.6)
(2, 0.0, 0)
(3, 10.1, 10.8)
(4, 0.0, 0)
WeightedKappaLoss: Median + IC95
[0.17540984]
[-0.01072994  0.46349899]


True

#### IMSize=320

In [17]:
modelname = "EfficientNetB3_v3_im320softmax.08--11.24"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
eval_class_result_all2(new_model)
eval_model_distrib(new_model, Niter = 250, sample_size = 300)

class | sens | true-pos-rate
(0, 100.0, 73.2)
(1, 0.0, 0)
(2, 0.0, 0)
(3, 0.0, 0.0)
(4, 0.0, 0)
Median + IC95
[1.74779737]
[-0.80786069 13.81551054]


True

In [8]:
# RELU 1 
modelname = "EfficientNetB3_v3_im320.01--0.62"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})


# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

138/138 [==============================] - 50s 352ms/step
class | sens | true-pos-rate
(0, 99.7, 73.3)
(1, 0.0, 0)
(2, 0.0, 0.0)
(3, 1.8, 18.2)
(4, 5.8, 33.3)
WeightedKappaLoss: Median + IC95
[-9.53673862e-07]
[-0.03090974  0.23553222]


True

In [ ]:
# RELU 1 
modelname = "EfficientNetB3_v3_im320.26--0.58"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
eval_class_result_all3(new_model)
eval_model_distrib3(new_model, Niter = 250, sample_size = 300)

In [43]:
# Modelo con activación "relu" : RELU epoch 2S
modelname = "EfficientNetB3_v3_im320.05--0.43"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=64)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

69/69 [==============================] - 733s 11s/step
class | sens | true-pos-rate
(0, 99.4, 73.7)
(1, 0.0, 0.0)
(2, 0.0, 0)
(3, 0.0, 0)
(4, 17.4, 26.8)
WeightedKappaLoss: Median + IC95
[0.10385771]
[-0.03975178  0.3868664 ]


True

In [41]:
# Modelo con activación "relu" : RELU epoch 2S
modelname = "EfficientNetB3_v3_im320.02--6.48"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=64)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

69/69 [==============================] - 661s 10s/step
class | sens | true-pos-rate
(0, 100.0, 73.1)
(1, 0.0, 0)
(2, 0.0, 0)
(3, 0.0, 0)
(4, 0.0, 0)
WeightedKappaLoss: Median + IC95
[-9.53673862e-07]
[-9.53673862e-07 -9.53673862e-07]


True

In [40]:
# Modelo con activación "relu" 
modelname = "EfficientNetB3_v3_im320.10--0.50"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=120)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)


37/37 [==============================] - 678s 18s/step
class | sens | true-pos-rate
(0, 99.2, 74.3)
(1, 0.3, 1.3)
(2, 0.0, 0)
(3, 0.0, 0)
(4, 11.6, 38.5)
WeightedKappaLoss: Median + IC95
[0.11282661]
[-0.0054557   0.40786014]


True

In [46]:
# Modelo con activación "relu" 
modelname = "EfficientNetB3_v3_im320.30--0.58"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)


138/138 [==============================] - 745s 5s/step
class | sens | true-pos-rate
(0, 98.9, 74.6)
(1, 0.6, 2.1)
(2, 0.0, 0)
(3, 5.5, 25.0)
(4, 4.7, 33.3)
WeightedKappaLoss: Median + IC95
[0.12280444]
[-0.00344978  0.37578494]


True

In [48]:
# Modelo con activación "relu" 
modelname = "EfficientNetB3_v3_im320.35--0.60"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)


138/138 [==============================] - 743s 5s/step
class | sens | true-pos-rate
(0, 98.9, 74.6)
(1, 0.6, 2.1)
(2, 0.0, 0)
(3, 5.5, 22.2)
(4, 3.5, 33.3)
WeightedKappaLoss: Median + IC95
[0.12536908]
[-0.01075495  0.36435149]


True

In [55]:
# Modelo con activación "relu" 
modelname = "EfficientNetB3_v3_im320.45--0.60"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

138/138 [==============================] - 748s 5s/step
class | sens | true-pos-rate
(0, 98.8, 74.5)
(1, 0.6, 2.0)
(2, 0.0, 0)
(3, 3.7, 19.0)
(4, 3.5, 37.5)
WeightedKappaLoss: Median + IC95
[0.11162785]
[-0.00641648  0.34319431]


True

### Modelos EfficientNetB0

#### IMSize=150

In [8]:
#  Ultimo modelo con efficientNetB0 (256 N+16N)
modelname = "EfficientNetB0_Augm_im150.04--0.10"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Convertir modelo para que acepte input de 320
initmodelsize = 150
inputs = tf.keras.Input(shape=(320, 320, 3))
x = tf.keras.layers.Resizing(initmodelsize,initmodelsize)(inputs)
outputs = new_model(x, training=False)
finalmodel = tf.keras.Model(inputs, outputs)
#finalmodel.save(base_dir + 'models/PretrainedNetworks/'+ modelname + '_im320.h5') # if saved

# Evaluation:
pred_valid = finalmodel.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)


138/138 [==============================] - 85s 594ms/step
class | sens | true-pos-rate
(0, 83.2, 74.5)
(1, 0.0, 0)
(2, 0.0, 0)
(3, 0.0, 0)
(4, 50.0, 5.3)
WeightedKappaLoss: Median + IC95
[0.09512078]
[-0.07185967  0.30013445]


True

In [32]:
# Modelo de los iniciales.
modelname = "EfficientNetB0.53--0.55"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Convertir modelo para que acepte input de 320
initmodelsize = 150
inputs = tf.keras.Input(shape=(320, 320, 3))
x = tf.keras.layers.Resizing(initmodelsize,initmodelsize)(inputs)
outputs = new_model(x, training=False)
finalmodel = tf.keras.Model(inputs, outputs)
#finalmodel.save(base_dir + 'models/PretrainedNetworks/'+ modelname + '_im320.h5') # if saved

# Evaluation:
pred_valid = finalmodel.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)


138/138 [==============================] - 8s 56ms/step
class | sens | true-pos-rate
(0, 92.2, 76.4)
(1, 4.2, 3.1)
(2, 1.0, 28.0)
(3, 0.0, 0)
(4, 32.6, 36.8)
WeightedKappaLoss: Median + IC95
[0.31717581]
[0.05951617 0.68567601]


True

In [21]:
# Modelo con activación "relu" 
modelname = "EfficientNetB0_Augm_im224.34--0.59"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Convertir modelo para que acepte input de 320
initmodelsize = 224
inputs = tf.keras.Input(shape=(320, 320, 3))
x = tf.keras.layers.Resizing(initmodelsize,initmodelsize)(inputs)
outputs = new_model(x, training=False)
finalmodel = tf.keras.Model(inputs, outputs)
#finalmodel.save(base_dir + 'models/PretrainedNetworks/'+ modelname + '_im320.h5') # if saved

# Evaluation:
pred_valid = finalmodel.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)


138/138 [==============================] - 128s 916ms/step
class | sens | true-pos-rate
(0, 98.2, 74.0)
(1, 1.0, 2.9)
(2, 0.0, 0)
(3, 0.0, 0)
(4, 8.1, 25.0)
WeightedKappaLoss: Median + IC95
[0.07636344]
[-0.02837352  0.28922281]


True

In [11]:
# Modelo con activación "relu" 
modelname = "EfficientNetB0_lrscheduled.90--0.54_im320"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

138/138 [==============================] - 9s 56ms/step
class | sens | true-pos-rate
(0, 78.8, 80.9)
(1, 21.7, 7.8)
(2, 9.5, 33.2)
(3, 0.0, 0)
(4, 50.0, 21.7)
WeightedKappaLoss: Median + IC95
[0.5484539]
[0.16418184 0.93576103]


True

In [46]:
# Modelo con activación "relu" 
modelname = "EfficientNetB0_lrscheduled_wsample.155--0.49"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Convertir modelo para que acepte input de 320
initmodelsize = 150
inputs = tf.keras.Input(shape=(320, 320, 3))
x = tf.keras.layers.Resizing(initmodelsize,initmodelsize)(inputs)
outputs = new_model(x, training=False)
finalmodel = tf.keras.Model(inputs, outputs)
#finalmodel.save(base_dir + 'models/PretrainedNetworks/'+ modelname + '_im320.h5') # if saved

# Evaluation:
pred_valid = finalmodel.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)


138/138 [==============================] - 9s 55ms/step
class | sens | true-pos-rate
(0, 60.9, 82.1)
(1, 17.3, 5.8)
(2, 25.6, 25.7)
(3, 0.0, 0)
(4, 59.3, 12.5)
WeightedKappaLoss: Median + IC95
[0.44098826]
[0.19381849 0.77395185]


True

**Best Model sent**

In [7]:
# Modelo con activación "relu" 
modelname = "EfficientNetB0_lrscheduled.86--0.53_im320"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)

138/138 [==============================] - 12s 57ms/step
class | sens | true-pos-rate
(0, 77.0, 81.2)
(1, 22.7, 7.5)
(2, 11.2, 32.1)
(3, 0.0, 0)
(4, 46.5, 24.1)
WeightedKappaLoss: Median + IC95
[0.54878539]
[0.23858967 0.91320414]


True

In [7]:
modelname = "EfficientNetB0_lrscheduled_.06--0.58_im320"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)

In [10]:
# Modelo con activación "relu" 
modelname = "EfficientNetB0_lrscheduled2_.02--0.58_im320"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

138/138 [==============================] - 9s 57ms/step
class | sens | true-pos-rate
(0, 86.1, 78.0)
(1, 12.8, 5.5)
(2, 3.6, 39.3)
(3, 0.0, 0)
(4, 19.8, 27.4)
WeightedKappaLoss: Median + IC95
[0.35385622]
[0.08395287 0.66618527]


True

In [7]:
# Modelo con activación "relu" 
modelname = "EfficientNetB3_v3_im320.47--0.62"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

138/138 [==============================] - 57s 354ms/step
class | sens | true-pos-rate
(0, 98.8, 74.5)
(1, 0.6, 2.0)
(2, 0.0, 0)
(3, 3.7, 17.4)
(4, 3.5, 33.3)
WeightedKappaLoss: Median + IC95
[0.10094289]
[-0.00599269  0.30333924]


True

In [12]:
# Modelo con activación "relu" 
modelname = "EfficientNetB3_v3_im320.68--0.65"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

138/138 [==============================] - 59s 388ms/step
class | sens | true-pos-rate
(0, 98.8, 74.6)
(1, 0.6, 2.0)
(2, 0.0, 0)
(3, 5.5, 26.1)
(4, 4.7, 36.4)
WeightedKappaLoss: Median + IC95
[0.12694184]
[-0.01702057  0.37412501]


True

In [ ]:
# Modelo con activación "relu" 
modelname = "EfficientNetB3_v3_im320.68--0.65"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

In [15]:
# Modelo con activación "relu" 
modelname = "EfficientNetB3_v3_im320.70--0.60"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

138/138 [==============================] - 59s 388ms/step
class | sens | true-pos-rate
(0, 98.8, 74.6)
(1, 0.6, 2.0)
(2, 0.0, 0)
(3, 5.5, 25.0)
(4, 4.7, 40.0)
WeightedKappaLoss: Median + IC95
[0.1266206]
[-0.00632462  0.39654783]


True

In [11]:
# Modelo con activación "relu" 
modelname = "EfficientNetB0_Augm_im224.06--0.51_im320"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

class | sens | true-pos-rate
(0, 99.3, 73.6)
(1, 0.0, 0.0)
(2, 0.0, 0)
(3, 0.9, 5.6)
(4, 9.3, 25.8)
WeightedKappaLoss: Median + IC95
[0.04785122]
[-0.03685377  0.34171506]


True

In [29]:
# Modelo Ensamblado
modelname = "EfficientNetB3_v3_im320.82--0.66"
new_model = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

138/138 [==============================] - 734s 5s/step
class | sens | true-pos-rate
(0, 98.9, 74.5)
(1, 0.6, 2.0)
(2, 0.0, 0)
(3, 4.6, 21.7)
(4, 4.7, 44.4)
WeightedKappaLoss: Median + IC95
[0.116194]
[-0.00612102  0.38696356]


True

## Modelos Ensamblados w Automatic Learning

In [30]:
# Modelo con activación "relu" 
modelname = "ModelEns1_im320320.03--0.34"
new_model = tf.keras.models.load_model(base_dir + 'models/EnsembledModel/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

138/138 [==============================] - 439s 3s/step
class | sens | true-pos-rate
(0, 98.8, 74.6)
(1, 0.6, 2.0)
(2, 0.0, 0)
(3, 5.5, 26.1)
(4, 4.7, 36.4)
WeightedKappaLoss: Median + IC95
[0.1277698]
[-0.00588814  0.3914008 ]


True

In [7]:
# Modelo Ensamblado1
modelname = "ModelEns1_im320320.01--0.36"
new_model = tf.keras.models.load_model(base_dir + 'models/EnsembledModel/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

138/138 [==============================] - 55s 336ms/step
class | sens | true-pos-rate
(0, 98.8, 74.6)
(1, 0.6, 2.0)
(2, 0.0, 0)
(3, 5.5, 26.1)
(4, 4.7, 36.4)
WeightedKappaLoss: Median + IC95
[0.11638458]
[-0.00577723  0.40232876]


True

In [ ]:
# Modelo Ensamblado1
modelname = "ModelEns1_im320320.04--0.33"
new_model = tf.keras.models.load_model(base_dir + 'models/EnsembledModel/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid = new_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)

## Modelos Ensamblados w Manual weigthing

1. Average model

In [9]:
# Modelo ensamblado manualmente:
modelname1 = "EfficientNetB0_lrscheduled.86--0.53_im320"
new_model1 = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

modelname2 = "EfficientNetB3_v3_im320.82--0.66"
new_model2 = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})


image_width = 320
image_height = image_width
model_1 = new_model1
model_1 = Model(inputs=model_1.inputs,
                outputs=model_1.outputs,
                name='Modelo1')
model_2 = new_model2
model_2 = Model(inputs=model_2.inputs,
                outputs=model_2.outputs,
                name='Modelo2')
models = [model_1, model_2]
model_input = tf.keras.layers.Input(shape=(image_width, image_height, 3))
model_outputs = [model(model_input) for model in models]
ensemble_output = tf.keras.layers.Average()(model_outputs)
ensemble_model = Model(inputs=model_input, outputs=ensemble_output, name='ensemble')
ensemble_model.save(base_dir + 'models/EnsembledModel/EnsMan_'+ modelname1 + "_" + modelname2 +'.h5')


# Evaluation:
pred_valid = ensemble_model.predict(X_valid, verbose=1, batch_size=32)
y_pre_valid = np.argmax(pred_valid, axis = 1)

eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 250, sample_size = 300)

138/138 [==============================] - 56s 376ms/step
class | sens | true-pos-rate
(0, 92.9, 77.8)
(1, 7.0, 6.9)
(2, 5.7, 37.3)
(3, 0.0, 0)
(4, 37.2, 24.1)
WeightedKappaLoss: Median + IC95
[0.45377935]
[0.13290523 0.83454523]


True

2. Weigthed average model

2.1. First ensembled model B0 + B3

In [8]:
# Modelo1 ensamblado con pesos:
modelname1 = "EfficientNetB0_lrscheduled.86--0.53_im320"
new_model1 = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname1 + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

modelname2 = "EfficientNetB3_v3_im320.82--0.66"
new_model2 = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname2 + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Evaluation:
pred_valid1 = new_model1.predict(X_valid, verbose=1, batch_size=32)
pred_valid2 = new_model2.predict(X_valid, verbose=1, batch_size=32)

138/138 [==============================] - 49s 337ms/step


In [14]:
pred_valid = pred_valid1*1 + pred_valid2*0
y_pre_valid = np.argmax(pred_valid, axis = 1)
eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)

class | sens | true-pos-rate
(0, 77.0, 81.2)
(1, 22.7, 7.5)
(2, 11.2, 32.1)
(3, 0.0, 0)
(4, 46.5, 24.1)
WeightedKappaLoss: Median + IC95
[0.54266515]
[0.23816063 0.9060863 ]


True

In [19]:
pred_valid = pred_valid1*0.6 + pred_valid2*0.4
y_pre_valid = np.argmax(pred_valid, axis = 1)
eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)

class | sens | true-pos-rate
(0, 77.6, 81.0)
(1, 23.0, 7.8)
(2, 11.0, 32.3)
(3, 0.0, 0)
(4, 46.5, 24.7)
WeightedKappaLoss: Median + IC95
[0.56114239]
[0.21882511 0.98428618]


True

2.2 Otro modelo

In [8]:
modelname1 = "EfficientNetB0_lrscheduled.86--0.53_im320"
new_model1 = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname1 + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

modelname2 = "EfficientNetB0_lrscheduled_.06--0.58"
new_model2 = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname2 + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

# Convertir modelo2 para que acepte input de 320
initmodelsize = 150
inputs = tf.keras.Input(shape=(320, 320, 3))
x = tf.keras.layers.Resizing(initmodelsize,initmodelsize)(inputs)
outputs = new_model2(x, training=False)
model2_final = tf.keras.Model(inputs, outputs)
#finalmodel.save(base_dir + 'models/PretrainedNetworks/'+ modelname + '_im320.h5') # if saved


# Evaluation:
pred_valid1 = new_model1.predict(X_valid, verbose=1, batch_size=32)
pred_valid2 = model2_final.predict(X_valid, verbose=1, batch_size=32)

138/138 [==============================] - 9s 56ms/step


In [9]:
weigth1 = 50
weigth2 = 50
# Obtain combination 
pred_valid = pred_valid1*weigth1/100 + pred_valid2*weigth2/100
y_pre_valid = np.argmax(pred_valid, axis = 1)
eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 5000, sample_size = 300)

class | sens | true-pos-rate
(0, 89.2, 77.7)
(1, 7.7, 4.4)
(2, 2.7, 37.5)
(3, 0.0, 0)
(4, 38.4, 28.2)
WeightedKappaLoss: Median + IC95
[0.41030999]
[0.11979033 0.81239852]


True

In [43]:
weigth1 = 100
weigth2 = 0
# Obtain combination 
pred_valid = pred_valid1*weigth1/100 + pred_valid2*weigth2/100
y_pre_valid = np.argmax(pred_valid, axis = 1)
eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 1000, sample_size = 300)

class | sens | true-pos-rate
(0, 77.0, 81.2)
(1, 22.7, 7.5)
(2, 11.2, 32.1)
(3, 0.0, 0)
(4, 46.5, 24.1)
WeightedKappaLoss: Median + IC95
[0.56257081]
[0.32187698 0.82463332]


True

In [44]:
weigth1 = 95
weigth2 = 5
# Obtain combination 
pred_valid = pred_valid1*weigth1/100 + pred_valid2*weigth2/100
y_pre_valid = np.argmax(pred_valid, axis = 1)
eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 1000, sample_size = 300)

class | sens | true-pos-rate
(0, 77.2, 81.2)
(1, 22.7, 7.5)
(2, 11.0, 31.9)
(3, 0.0, 0)
(4, 46.5, 24.1)
WeightedKappaLoss: Median + IC95
[0.56102738]
[0.33247174 0.82119677]


True

2.3 Otro modelo

Modelo 1

In [11]:
modelname1 = "EfficientNetB0_lrscheduled.86--0.53"
new_model1 = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname1 + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

 
# Convertir modelo2 para que acepte input de 320
initmodelsize = 150
inputs = tf.keras.Input(shape=(320, 320, 3))
x = tf.keras.layers.Resizing(initmodelsize,initmodelsize)(inputs)
outputs = new_model1(x, training=False)
model1_final = tf.keras.Model(inputs, outputs)
#finalmodel.save(base_dir + 'models/PretrainedNetworks/'+ modelname + '_im320.h5') # if saved

# Predict and save
pred_valid1 = model1_final.predict(X_valid, verbose=1, batch_size=32)

138/138 [==============================] - 9s 56ms/step


Modelo2 

In [12]:
modelname2 = "EfficientNetB0_Augm_im224.33--0.60"
new_model2 = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelname2 + '.h5',
    custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
    'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

 
# Convertir modelo2 para que acepte input de 320
initmodelsize = 224
inputs = tf.keras.Input(shape=(320, 320, 3))
x = tf.keras.layers.Resizing(initmodelsize,initmodelsize)(inputs)
#x = tf.keras.layers.Rescaling(scale=1/255)(x)
outputs = new_model2(x, training=False)
model2_final = tf.keras.Model(inputs, outputs)
#finalmodel.save(base_dir + 'models/PretrainedNetworks/'+ modelname + '_im320.h5') # if saved

# Predict and save
pred_valid2 = model2_final.predict(X_valid, verbose=1, batch_size=32)


138/138 [==============================] - 14s 94ms/step


Testing combinations

In [17]:
pred_valid1

array([[8.2142409e-15, 1.0000000e+00, 1.4532386e-30, 0.0000000e+00,
        0.0000000e+00],
       [1.7557878e-34, 1.0000000e+00, 1.8495453e-24, 0.0000000e+00,
        0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00],
       ...,
       [9.9999845e-01, 1.5644404e-06, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, 0.0000000e+00,
        0.0000000e+00],
       [1.0000000e+00, 3.5004891e-08, 3.8741978e-32, 0.0000000e+00,
        0.0000000e+00]], dtype=float32)

In [25]:
weigth1 = 100
weigth2 = 0
# Obtain combination 
pred_valid = pred_valid1*weigth1/100 + pred_valid2*weigth2/100
y_pre_valid = np.argmax(pred_valid, axis = 1)
eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 1000, sample_size = 5000)

class | sens | true-pos-rate
(0, 77.0, 81.2)
(1, 22.7, 7.5)
(2, 11.2, 32.1)
(3, 0.0, 0)
(4, 46.5, 24.1)
WeightedKappaLoss: Median + IC95
[0.5624668]
[0.50311133 0.62453474]


True

In [14]:
weigth1 = 100
weigth2 = 0
# Obtain combination 
pred_valid = pred_valid1*weigth1/100 + pred_valid2*weigth2/100
y_pre_valid = np.argmax(pred_valid, axis = 1)
eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 1000, sample_size = 5000)

class | sens | true-pos-rate
(0, 77.0, 81.2)
(1, 22.7, 7.5)
(2, 11.2, 32.1)
(3, 0.0, 0)
(4, 46.5, 24.1)
WeightedKappaLoss: Median + IC95
[0.56283718]
[0.50023053 0.62087616]


True

In [26]:
weigth1 = 90
weigth2 = 10
# Obtain combination 
pred_valid = pred_valid1*weigth1/100 + pred_valid2*weigth2/100
y_pre_valid = np.argmax(pred_valid, axis = 1)
eval_class_result_all3(Y_valid, y_pre_valid)
eval_model_distrib3(Y_valid, y_pre_valid, Niter = 1000, sample_size = 5000)

class | sens | true-pos-rate
(0, 77.2, 81.2)
(1, 22.7, 7.5)
(2, 11.2, 32.1)
(3, 0.0, 0)
(4, 46.5, 24.1)
WeightedKappaLoss: Median + IC95
[0.56425679]
[0.50669972 0.62739525]


True

Ensemble of various models

In [18]:
modelnames = ["EfficientNetB0_lrscheduled.86--0.53","EfficientNetB3_v3_im320.82--0.66","EfficientNetB0_Augm_im224.33--0.60"]
weigths = [60,30,10]
sizeinput = [150, 320, 224]

predictions = list()
wave_pred =  0
for this_i in range(len(modelnames)):
    #modelnamei = "EfficientNetB0_lrscheduled.86--0.53"
    modelnamei = modelnames[this_i]
    print("Model: ", modelnamei)
    thismodeli = tf.keras.models.load_model(base_dir + 'models/PretrainedNetworks/' + modelnamei + '.h5',
        custom_objects={'WeightedCategoricalCrossentropy': WeightedCategoricalCrossentropy(cost_mat=cost_mat),
        'WeightedCategoricalAccuracy': WeightedCategoricalAccuracy(cost_mat=cost_mat)})

    
    # Convertir modelo2 para que acepte input de 320
    initmodelsize = sizeinput[this_i]
    finalmodel = thismodeli
    if initmodelsize<320:
        inputs = tf.keras.Input(shape=(320, 320, 3))
        x = tf.keras.layers.Resizing(initmodelsize,initmodelsize)(inputs)        
        outputs = thismodeli(x, training=False)
        finalmodel = tf.keras.Model(inputs, outputs)
    #finalmodel.save(base_dir + 'models/PretrainedNetworks/'+ modelnamei + '_im320.h5') # if saved

    # Predict and save
    this_prediction = finalmodel.predict(X_valid, verbose=1, batch_size=32)
    predictions.append(this_prediction)

    wave_pred = this_prediction*(weigths[this_i]/np.sum(weigths))

    # Show results
    y_pre_valid = np.argmax(this_prediction, axis = 1)
    eval_class_result_all3(Y_valid, y_pre_valid)
    eval_model_distrib3(Y_valid, y_pre_valid, Niter = 200, sample_size = 5000)

Model:  EfficientNetB0_lrscheduled.86--0.53
138/138 [==============================] - 65s 462ms/step
class | sens | true-pos-rate
(0, 77.0, 81.2)
(1, 22.7, 7.5)
(2, 11.2, 32.1)
(3, 0.0, 0)
(4, 46.5, 24.1)
WeightedKappaLoss: Median + IC95
[0.55950096]
[0.50205465 0.62593307]
Model:  EfficientNetB3_v3_im320.82--0.66
138/138 [==============================] - 431s 3s/step
class | sens | true-pos-rate
(0, 98.9, 74.5)
(1, 0.6, 2.0)
(2, 0.0, 0)
(3, 4.6, 21.7)
(4, 4.7, 44.4)
WeightedKappaLoss: Median + IC95
[0.13156114]
[0.09698712 0.1617627 ]
Model:  EfficientNetB0_Augm_im224.33--0.60
138/138 [==============================] - 128s 920ms/step
class | sens | true-pos-rate
(0, 98.2, 74.0)
(1, 1.0, 2.9)
(2, 0.0, 0)
(3, 0.0, 0)
(4, 8.1, 25.0)
WeightedKappaLoss: Median + IC95
[0.09104246]
[0.0615885  0.12613663]


In [ ]:
weigths = weigths
wave_pred = 0
for this_i in range(len(predictions)):
    wave_pred = wave_pred + predictions[this_i]*(weigths[this_i]/np.sum(weigths))

y_pre_valid = np.argmax(wave_pred, axis = 1)
eval_class_result_all3(Y_valid, y_pre_valid)

eval_model_distrib3(Y_valid, y_pre_valid, Niter = 200, sample_size = 5000)

class | sens | true-pos-rate
(0, 77.3, 81.1)
(1, 23.0, 7.7)
(2, 11.0, 32.3)
(3, 0.0, 0)
(4, 46.5, 24.5)
WeightedKappaLoss: Median + IC95
[0.55646187]
[0.49778648 0.61417418]


True

In [ ]:
weigths = [0,0,100]
wave_pred = 0
for this_i in range(len(predictions)):
    wave_pred = wave_pred + predictions[this_i]*(weigths[this_i]/np.sum(weigths))

y_pre_valid = np.argmax(wave_pred, axis = 1)
eval_class_result_all3(Y_valid, y_pre_valid)

eval_model_distrib3(Y_valid, y_pre_valid, Niter = 1000, sample_size = 5000)

class | sens | true-pos-rate
(0, 98.2, 74.0)
(1, 1.0, 2.9)
(2, 0.0, 0)
(3, 0.0, 0)
(4, 8.1, 25.0)
WeightedKappaLoss: Median + IC95
[0.08873658]
[0.05768638 0.12445675]


True